In [1]:
#!/bin/python3

#%%
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import keras_tuner as kt
import sklearn as skl
from sklearn.model_selection import train_test_split

print('Tensorflow version: ' + str(tf.__version__))
print('Keras version: ' + str(keras.__version__))
print('KerasTuner version: ' + str(kt.__version__))
print('Scikit-learn version: ' + str(skl.__version__))

Tensorflow version: 2.15.0
Keras version: 3.0.5
KerasTuner version: 1.4.6
Scikit-learn version: 1.3.0


In [2]:
# load the datasets
path = './datasets/GTSRB/'

print('Loading datasets...')
training = np.load(os.path.join(path, 'training.npz'), allow_pickle=True)
testing = np.load(os.path.join(path, 'testing.npz'), allow_pickle=True)

X_train = training['X']
y_train = training['y']

X_test = testing['X']
y_test = testing['y']

Loading datasets...


In [3]:
# converting datatypes
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

In [4]:
# split training data into training and validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2,
                                                      shuffle=True)

In [5]:
# augment data

def augment_data(X):
    layers = [
        tf.keras.layers.RandomRotation(1/8), # 45 degrees, either cw/ccw
        tf.keras.layers.RandomTranslation(0.1, 0.1) # shift w/h by up to 10%
    ]

    X_augmented = []

    for x in X:
        for layer in layers:
            x = layer(x)
        
        X_augmented.append(x)
    
    return np.asarray(X_augmented).astype('float32')

print('Augmenting training and validation data...')
X_train = tf.py_function(func=augment_data, inp=[X_train], Tout=tf.float32)
X_valid = tf.py_function(func=augment_data, inp=[X_valid], Tout=tf.float32)

Augmenting training and validation data...


In [6]:
# build model
print('Searching and building a model...')

def build_model(hp):
    activation_functions = ['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh',
                            'selu', 'elu', 'exponential', 'leaky_relu', 'relu6', 'silu',
                            'gelu', 'hard_sigmoid', 'log_softmax', 'mish', 'linear']
    filters_0 = hp.Int('filters_0', 64, 256, 32)
    filters_1 = hp.Int('filters_1', 128, 384, 32)
    units_lc = hp.Int('units_lc', np.min([filters_1 - 32, 96]), np.max([filters_1 + 32, 128]), 8)
    activation_lc = hp.Choice('activation_lc', activation_functions)
    dropout_l = hp.Float('dropout_l', 0.0, 0.5)
    activation_final = hp.Choice('activation_final', activation_functions)

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(50, 50, 3)),
        tf.keras.layers.Conv2D(filters_0, kernel_size=(5, 5)),
        tf.keras.layers.MaxPooling2D(pool_size=(5, 3)),
        tf.keras.layers.Conv2D(filters_1, kernel_size=(5, 5)),
        tf.keras.layers.MaxPooling2D(pool_size=(5, 3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units_lc, activation=activation_lc),
        tf.keras.layers.Dropout(dropout_l),
        tf.keras.layers.Dense(43, activation=activation_final)
    ])

    loss_from_logits = True
    if activation_final in ['sigmoid', 'softmax']:
        loss_from_logits = False

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=loss_from_logits),
                  metrics=['accuracy'])
    return model

tuner = kt.BayesianOptimization(build_model,
                        project_name='kt_convolutional_model',
                        objective='val_accuracy',
                        max_trials=20,
                        seed=42
                        )
tuner.search(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid), callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
])
tuner.results_summary()
model = tuner.get_best_models()[0]

model.summary()

Trial 20 Complete [00h 00m 35s]
val_accuracy: 0.9622793197631836

Best val_accuracy So Far: 0.9642857313156128
Total elapsed time: 00h 19m 28s
Results summary
Results in .\kt_convolutional_model
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 18 summary
Hyperparameters:
filters_0: 128
filters_1: 384
units_lc: 144
activation_lc: relu6
dropout_l: 0.07800932022121826
activation_final: softmax
Score: 0.9642857313156128

Trial 12 summary
Hyperparameters:
filters_0: 96
filters_1: 384
units_lc: 152
activation_lc: softsign
dropout_l: 0.35549113389814224
activation_final: mish
Score: 0.9626805782318115

Trial 19 summary
Hyperparameters:
filters_0: 64
filters_1: 256
units_lc: 128
activation_lc: relu6
dropout_l: 0.3630456668613308
activation_final: linear
Score: 0.9622793197631836

Trial 15 summary
Hyperparameters:
filters_0: 128
filters_1: 256
units_lc: 128
activation_lc: silu
dropout_l: 0.28785810406821527
activation_final: sigmoid
Score: 0.9578651785850525

Trial 

c:\Users\leona\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 128)    │         9,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 9, 15, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 5, 11, 384)     │     1,229,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 3, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 144)            │       166,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 144)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 43)             │         6,235 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,411,179 (5.38 MB)

 Trainable params: 1,411,179 (5.38 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# fit model
print('Fitting model...')
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                                     restore_best_weights=True)
])

Fitting model...
Epoch 1/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 17s 50ms/step - accuracy: 0.9697 - loss: 0.0998 - val_accuracy: 0.9298 - val_loss: 0.2288
Epoch 2/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.9617 - loss: 0.1196 - val_accuracy: 0.9358 - val_loss: 0.2309
Epoch 3/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.9673 - loss: 0.0989 - val_accuracy: 0.9314 - val_loss: 0.2328
Epoch 4/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.9609 - loss: 0.1136 - val_accuracy: 0.9402 - val_loss: 0.1906
Epoch 5/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.9595 - loss: 0.1242 - val_accuracy: 0.9498 - val_loss: 0.1676
Epoch 6/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.9733 - loss: 0.0824 - val_accuracy: 0.9478 - val_loss: 0.1885
Epoch 7/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.9800 - loss: 0.0597 - val_accuracy: 0.9366 - val_loss: 0.2013
Epoch 8/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.9582 

In [8]:
# save model
model.save('convolutional_model.keras')

In [9]:
# load model
model = tf.keras.models.load_model('convolutional_model.keras')

In [10]:
# testing
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

121/121 - 2s - 17ms/step - accuracy: 0.9409 - loss: 0.2624
Test accuracy: 0.9409008026123047
Test accuracy: 0.9409008026123047
